In [2]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasis1.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
age = []
fc = []
sex = []
race = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for subtask in basis:
    m = re.search('([^-]+)-(.*)', subtask)
    sub = m.group(1)
    task = m.group(2)
    a = demo['age_at_cnb'][sub]
    age.append(a)
    s = demo['Sex'][sub] == 'M'
    r = demo['Race'][sub] == 'AA'
    sex.append(s)
    race.append(r)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    fc.append(p)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
age = np.array(age)
fc = np.stack(fc)
sex = np.array(sex).astype('int')
race = np.array(race).astype('int')

print([a.shape for a in [thetas, jitter, age, sex, race, fc]])
print(np.mean(age))

[(3849, 1, 264), (3849, 1, 264), (3849,), (3849,), (3849,), (3849, 34716)]
14.398285268901013


In [15]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def cossinjit(thetas, jitter):
    t = thetas.squeeze()
    s = np.sin(t)
    c = np.cos(t)
    sc = s*c
    j = jitter.squeeze()
    return np.concatenate([s,c,sc],axis=1)
    
feat = cossinjit(thetas, jitter)

print(feat.shape)

(3849, 792)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

idcs = np.random.permutation(fc.shape[0])
x = feat[idcs[:500]]
y = age[idcs[:500]]

xtr, xt, ytr, yt = train_test_split(x, y, train_size=0.8)

mu = np.mean(ytr)
ytr -= mu
yt -= mu

reg = Ridge(alpha=1).fit(xtr, ytr)
yhat = reg.predict(xt)
print(rmse(yhat, yt))
print(rmse(0, yt))

5.131180351680168
3.340303129956921
